In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch import exceptions
from elastic_transport import ObjectApiResponse

In [2]:
es = Elasticsearch(["http://192.168.10.50:9200"])

In [170]:
ess = Elasticsearch(["http://192.168.10.50:10000"])

In [172]:
ess.ping()

False

In [3]:
es.info()

ObjectApiResponse({'name': 'loovee-1', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'oQQ9Gx1pRfCoUGB4McKUeg', 'version': {'number': '8.8.2', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '98e1271edf932a480e4262a471281f1ee295ce6b', 'build_date': '2023-06-26T05:16:16.196344851Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [4]:
es.ping()

True

In [8]:
es.indices.get_alias().keys()

/tmp/ipykernel_77299/129244569.py:1: ElasticsearchWarning: this request accesses system indices: [.security-7], but in a future major version, direct access to system indices will be prevented by default
  es.indices.get_alias().keys()


dict_keys(['test-index', '.security-7', 'test_index', 'my-index-000001'])

In [12]:
es.indices.exists(index="test-index")

HeadApiResponse(True)

In [14]:
es.indices.get_mapping().keys()

dict_keys(['my-index-000001', 'test-index', 'test_index'])

In [106]:
es.indices.get_mapping()['test-index']

{'mappings': {'dynamic': 'true',
  'numeric_detection': False,
  'properties': {'age': {'type': 'byte'},
   'audit_state': {'type': 'byte'},
   'avatar': {'type': 'text'},
   'avatar_certify': {'type': 'byte'},
   'birthday': {'type': 'keyword'},
   'blood_type': {'type': 'keyword'},
   'career': {'type': 'text',
    'analyzer': 'ik_max_word',
    'search_analyzer': 'ik_smart'},
   'education': {'type': 'byte'},
   'education_certify': {'type': 'byte'},
   'ethnic': {'type': 'byte'},
   'favorite_mate': {'type': 'text',
    'analyzer': 'ik_max_word',
    'search_analyzer': 'ik_smart'},
   'gender': {'type': 'keyword'},
   'height': {'type': 'short'},
   'hometown_city': {'type': 'keyword'},
   'hometown_province': {'type': 'keyword'},
   'horoscope': {'type': 'keyword'},
   'income': {'type': 'byte'},
   'info_finish_percent': {'type': 'byte'},
   'interest': {'type': 'text',
    'analyzer': 'ik_max_word',
    'search_analyzer': 'ik_smart'},
   'love_concept': {'type': 'text',
    'ana

In [107]:
es.indices.get_settings()['test-index']

{'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
   'number_of_shards': '2',
   'provided_name': 'test-index',
   'creation_date': '1688698144685',
   'number_of_replicas': '2',
   'uuid': 'eHJHN8wjQ6OfAWtu50Yptg',
   'version': {'created': '8080299'}}}}

In [17]:
es.indices.get_settings().keys()

dict_keys(['test-index', 'test_index', 'my-index-000001'])

In [25]:
for x in es.search(index="test_index"):
    print(type(x))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [192]:
index = "test_index"
id = 200
document = {"nick":"我是一条新样本","gender":"male","marry_state":0,"height":170, "weight":60}
try:
    res = es.index(index=index, id=id, document=document)
    print(res)
except exceptions.BadRequestError as e:
    print(e.message)
    print(e.status_code)

{'_index': 'test_index', '_id': '200', '_version': 2, 'result': 'updated', '_shards': {'total': 3, 'successful': 1, 'failed': 0}, '_seq_no': 148, '_primary_term': 1}


In [193]:
res = es.delete(index="test_index", id=200)
print(res)

{'_index': 'test_index', '_id': '200', '_version': 3, 'result': 'deleted', '_shards': {'total': 3, 'successful': 1, 'failed': 0}, '_seq_no': 149, '_primary_term': 1}


In [174]:
print(es.get(index="test_index", id=1000))

{'_index': 'test_index', '_id': '100', '_version': 1, '_seq_no': 146, '_primary_term': 1, 'found': True, '_source': {'nick': '我是一条新样本', 'gender': 'male', 'marry_state': 0, 'height': 170, 'weight': 60}}


In [179]:
print(es.exists(index="test_index", id=1000))

False


In [119]:
# 查询文档总数
es.indices.refresh(index="test_index")
es.cat.count(index="test_index", format="json")

ListApiResponse([{'epoch': '1688952616', 'timestamp': '01:30:16', 'count': '293'}])

In [27]:
# # 只显示test索引的age字段信息
print(es.search(index='test_index',filter_path=['hits.hits._source.age']))

{'hits': {'hits': [{'_source': {'age': 36}}, {'_source': {'age': 48}}, {'_source': {'age': 48}}, {'_source': {'age': 49}}, {'_source': {'age': 39}}, {'_source': {'age': 37}}, {'_source': {'age': 30}}, {'_source': {'age': 40}}, {'_source': {'age': 47}}, {'_source': {'age': 47}}]}}


In [122]:
# 查询test索引中，从序号为1的位置查询两条数据
res = es.search(index='test_index', from_=1, size=3)
for k,v in res.items():
    print(f"{k}：{v}")

took：3
timed_out：False
_shards：{'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}
hits：{'total': {'value': 293, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '177929372', '_score': 1.0, '_source': {'nick': '不离不弃ever', 'gender': 'male', 'marry_state': 0, 'height': 175, 'weight': 99, 'education': 7, 'residence_city': '西城', 'age': 48, 'interest': '[网络, 读书, 体育运动, 旅游, 中国菜, 健康食品, 励志与成功, 行业发展, 惬意海岛, 繁华都市, 秀美山水, 江河大川, 小桥流水人家]', 'self_introduce': '北京名校本硕毕业，京户，西城有房（无贷）。别人评价人品好、靠谱、综合素质高。包容、幽默、孝顺、稳重、诚信，有责任心。休息时看书，健身，听音乐。注重饮食，健康，不吸烟不好酒，没有任何不良嗜好。自身条件不差，之所以至今未婚是因为当年从外地考到帝都一路打拼，主要精力放在学业工作上，也因为婚姻大事关乎一生，甚至下一代，不想草率结婚，但也不是挑剔，只看缘分。***警告：本人只为婚姻，非诚勿扰远离。各种托、中介、理财、博彩、传销、支教资助等形形色色骗子被本人发现一律报警。***告诉你一个防骗秘诀：只要提到钱让你掏腰包的就是骗子，无论什么理由！！', 'income': 2.0, 'horoscope': '天秤座', 'blood_type': 'O', 'profession': 9.0, 'ethnic': 56.0, 'university': '清华大学'}}, {'_index': 'test_index', '_id': '173975328', '_score': 1.0, '_source': {'nick': 'Mr Rt158', 'gender': 'male', 'marry_state': 0, 'heigh

In [123]:
print("成功获取：", res['took'])
for per in res['hits']['hits']:
    print(per)

成功获取： 3
{'_index': 'test_index', '_id': '177929372', '_score': 1.0, '_source': {'nick': '不离不弃ever', 'gender': 'male', 'marry_state': 0, 'height': 175, 'weight': 99, 'education': 7, 'residence_city': '西城', 'age': 48, 'interest': '[网络, 读书, 体育运动, 旅游, 中国菜, 健康食品, 励志与成功, 行业发展, 惬意海岛, 繁华都市, 秀美山水, 江河大川, 小桥流水人家]', 'self_introduce': '北京名校本硕毕业，京户，西城有房（无贷）。别人评价人品好、靠谱、综合素质高。包容、幽默、孝顺、稳重、诚信，有责任心。休息时看书，健身，听音乐。注重饮食，健康，不吸烟不好酒，没有任何不良嗜好。自身条件不差，之所以至今未婚是因为当年从外地考到帝都一路打拼，主要精力放在学业工作上，也因为婚姻大事关乎一生，甚至下一代，不想草率结婚，但也不是挑剔，只看缘分。***警告：本人只为婚姻，非诚勿扰远离。各种托、中介、理财、博彩、传销、支教资助等形形色色骗子被本人发现一律报警。***告诉你一个防骗秘诀：只要提到钱让你掏腰包的就是骗子，无论什么理由！！', 'income': 2.0, 'horoscope': '天秤座', 'blood_type': 'O', 'profession': 9.0, 'ethnic': 56.0, 'university': '清华大学'}}
{'_index': 'test_index', '_id': '173975328', '_score': 1.0, '_source': {'nick': 'Mr Rt158', 'gender': 'male', 'marry_state': 0, 'height': 175, 'weight': 48, 'education': 7, 'residence_city': '西城', 'age': 48, 'interest': '[中国菜, 健康食品, 励志与成功, 投资理财, 人文社科, 自然科学, 什么都看, 惬意海岛, 繁华都市, 秀美山水, 历史遗迹, 江河大

In [114]:
for k,v in res.items():
    print(k,v)

took 2
timed_out False
_shards {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}
hits {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}


In [127]:
# 模糊查询（分词）
# 查询索引中，age字段为20的数据
res = es.search(index='test_index', query={'match':{'age':20}})
# 匹配失败
print(res)
res1 = es.search(index='test_index', query={'match':{'age':49}})
# 匹配成功
print(res1)

{'took': 2, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
{'took': 4, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '129852534', '_score': 1.0, '_source': {'nick': 'zeng', 'gender': 'male', 'marry_state': 0, 'height': 168, 'weight': 70, 'education': 7, 'residence_city': '贵阳', 'age': 49, 'interest': '[写作, 读书, 绘画, 中国菜, 文学, 政治与军事, 健康与养生, 投资理财, 人文社科, 喜剧, 战争, 纪录片, 政治事件, 理财投资, 图书馆/书店, 惬意海岛, 繁华都市, 广袤森林, 历史遗迹, 小桥流水人家]', 'self_introduce': '我是1974年生，硕士学历，本科学工科。研究生读中文，做的工作是与文字有关，平时最大的爱好就是画画，或做些插图设计。我在前几年买了一套二手房，环境好，旁边有一个依河建的湿地公园。有房贷，但公积金可以覆盖房贷。无车贷。我的三观算是比较正的，从2014年开始，每月向一个儿童基金会捐一百元，每月从我的卡上自动转过去。由于种种原因，错过了一些我爱的人和爱我的人。年龄不小了，可期望还在。所以，希望在这里能找到那个\\\\" 我的爱人\\\\"。', 'income': 1.0, 'horoscope': '双鱼座', 'profession': 11.0, 'hometown_p

In [128]:
print("匹配成功数:", res['hits']['total']['value'])
print("匹配成功数:", len(res['hits']['hits']))
for per in res['hits']['hits']:
    print(per)
print("匹配成功数:", res1['hits']['total']['value'])
print("匹配成功数:", len(res1['hits']['hits']))
for per in res1['hits']['hits']:
    print(per)

匹配成功数: 0
匹配成功数: 0
匹配成功数: 8
匹配成功数: 8
{'_index': 'test_index', '_id': '129852534', '_score': 1.0, '_source': {'nick': 'zeng', 'gender': 'male', 'marry_state': 0, 'height': 168, 'weight': 70, 'education': 7, 'residence_city': '贵阳', 'age': 49, 'interest': '[写作, 读书, 绘画, 中国菜, 文学, 政治与军事, 健康与养生, 投资理财, 人文社科, 喜剧, 战争, 纪录片, 政治事件, 理财投资, 图书馆/书店, 惬意海岛, 繁华都市, 广袤森林, 历史遗迹, 小桥流水人家]', 'self_introduce': '我是1974年生，硕士学历，本科学工科。研究生读中文，做的工作是与文字有关，平时最大的爱好就是画画，或做些插图设计。我在前几年买了一套二手房，环境好，旁边有一个依河建的湿地公园。有房贷，但公积金可以覆盖房贷。无车贷。我的三观算是比较正的，从2014年开始，每月向一个儿童基金会捐一百元，每月从我的卡上自动转过去。由于种种原因，错过了一些我爱的人和爱我的人。年龄不小了，可期望还在。所以，希望在这里能找到那个\\\\" 我的爱人\\\\"。', 'income': 1.0, 'horoscope': '双鱼座', 'profession': 11.0, 'hometown_province': '湖南', 'hometown_city': '娄底', 'ethnic': 56.0, 'university': '湘潭大学'}}
{'_index': 'test_index', '_id': '73619099', '_score': 1.0, '_source': {'nick': '孤独的心', 'gender': 'male', 'marry_state': 1, 'height': 168, 'weight': 54, 'education': 4, 'residence_city': '成都', 'age': 49, 'interest': '[旅游, 中国菜, 文学, 投资理财, 动作冒险, 悬疑推理,

In [125]:
res

ObjectApiResponse({'took': 5, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 8, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '129852534', '_score': 1.0, '_source': {'nick': 'zeng', 'gender': 'male', 'marry_state': 0, 'height': 168, 'weight': 70, 'education': 7, 'residence_city': '贵阳', 'age': 49, 'interest': '[写作, 读书, 绘画, 中国菜, 文学, 政治与军事, 健康与养生, 投资理财, 人文社科, 喜剧, 战争, 纪录片, 政治事件, 理财投资, 图书馆/书店, 惬意海岛, 繁华都市, 广袤森林, 历史遗迹, 小桥流水人家]', 'self_introduce': '我是1974年生，硕士学历，本科学工科。研究生读中文，做的工作是与文字有关，平时最大的爱好就是画画，或做些插图设计。我在前几年买了一套二手房，环境好，旁边有一个依河建的湿地公园。有房贷，但公积金可以覆盖房贷。无车贷。我的三观算是比较正的，从2014年开始，每月向一个儿童基金会捐一百元，每月从我的卡上自动转过去。由于种种原因，错过了一些我爱的人和爱我的人。年龄不小了，可期望还在。所以，希望在这里能找到那个\\\\" 我的爱人\\\\"。', 'income': 1.0, 'horoscope': '双鱼座', 'profession': 11.0, 'hometown_province': '湖南', 'hometown_city': '娄底', 'ethnic': 56.0, 'university': '湘潭大学'}}, {'_index': 'test_index', '_id': '73619099', '_score': 1.0, '_source': {'nick': '孤独的心',

In [145]:
# 查询所有
res = es.search(index="test_index",  query={'match_all':{}})
print(res)

{'took': 2, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 293, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '244389161', '_score': 1.0, '_source': {'nick': 'Panu', 'gender': 'male', 'marry_state': 0, 'height': 180, 'weight': 72, 'education': 8, 'residence_city': '海淀', 'age': 36, 'self_introduce': '我是芬兰数学家，在中科院工作. 我想找一个在北京工作和生活的女朋友.', 'income': 5.0, 'horoscope': '天秤座', 'blood_type': 'O', 'profession': 4.0, 'hometown_province': '其他'}}, {'_index': 'test_index', '_id': '177929372', '_score': 1.0, '_source': {'nick': '不离不弃ever', 'gender': 'male', 'marry_state': 0, 'height': 175, 'weight': 99, 'education': 7, 'residence_city': '西城', 'age': 48, 'interest': '[网络, 读书, 体育运动, 旅游, 中国菜, 健康食品, 励志与成功, 行业发展, 惬意海岛, 繁华都市, 秀美山水, 江河大川, 小桥流水人家]', 'self_introduce': '北京名校本硕毕业，京户，西城有房（无贷）。别人评价人品好、靠谱、综合素质高。包容、幽默、孝顺、稳重、诚信，有责任心。休息时看书，健身，听音乐。注重饮食，健康，不吸烟不好酒，没有任何不良嗜好。自身条件不差，之所以至今未婚是因为当年从外地考到帝都一路打拼，主要精力放在学业工作上，也因为

##### 模糊查询匹配

In [154]:
# keyword字段
res = es.search(index="test_index",  query={'match':{'nick':'我心'}})
print("A:", res)
res = es.search(index="test_index",  query={'match':{'nick':'我心飞翔'}})
print("B:", res)
res = es.search(index="test_index",  query={'match_phrase':{'nick':'我心'}})
print("C:", res)
res = es.search(index="test_index",  query={'match_phrase':{'nick':'飞翔'}})
print("D:", res)
res = es.search(index="test_index",  query={'match_phrase':{'nick':'我心飞翔'}})
print("E:", res)

A: {'took': 1, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
B: {'took': 1, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 4.591747, 'hits': [{'_index': 'test_index', '_id': '49575145', '_score': 4.591747, '_source': {'nick': '我心飞翔', 'gender': 'male', 'marry_state': 0, 'height': 173, 'weight': 90, 'education': 7, 'residence_city': '九龙坡', 'age': 47, 'interest': '[网络, 动物, 烹饪, 读书, 计算机, 中国菜, 健康食品, 励志与成功, 政治与军事, 哲学与宗教, 健康与养生, 自然科学, 动作冒险, 战争, 纪录片, 历史传记, 政治事件, 畅销书, 休闲生活, 行业发展, 饭店, 卡拉OK, 名山古刹, 风情名城, 高原雪域, 历史遗迹, 峻秀峡谷]', 'self_introduce': '真诚交友，非诚勿扰！介绍一下自己，我在一家国企工作，工作很忙，事情也很多，但相对来讲工作比较稳定，收入也还可以。这几年为了提升自己，又用周末时间重返校园攻读研究生硕士，所以这几年基本上是5+2、白+黑的状态，也耽误了婚姻大事。我属兔，农历是75年12月，公历是76年1月，个人有点小迷信，我希望最好能找属狗、猪、羊或其次是属兔、猴、虎的女朋友，同时我长得比较年轻，基本上看不出40多岁，所以希望女方年龄也不要超过38岁。我没有结过婚，也希望找个未婚的女性。异地恋也不太

In [155]:
# text字段
res = es.search(index="test_index",  query={'match':{'interest':'巧克力'}})
print("A:", res)
res = es.search(index="test_index",  query={'match':{'interest':'巧克力和甜点'}})
print("B:", res)
res = es.search(index="test_index",  query={'match_phrase':{'interest':'巧克力'}})
print("C:", res)
res = es.search(index="test_index",  query={'match_phrase':{'interest':'巧克力和甜点'}})
print("D:", res)


A: {'took': 5, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 16, 'relation': 'eq'}, 'max_score': 2.579227, 'hits': [{'_index': 'test_index', '_id': '13941921', '_score': 2.579227, '_source': {'nick': '洛陽塵', 'gender': 'female', 'marry_state': 0, 'height': 167, 'weight': 45, 'education': 4, 'residence_city': '深圳', 'age': 40, 'interest': '[中国菜, 巧克力和甜点, 励志与成功, 动漫与幽默, 动作冒险, 悬疑推理, 战争, 剧情, 电影院, 咖啡厅, 惬意海岛, 名山古刹, 高原雪域, 秀美山水, 历史遗迹]', 'self_introduce': '资料错了,巨蟹座。蓦然回首心戚戚，有缘的他你在哪？在熟人眼里我是个善良、温柔美丽的贤妻良母型女孩。在男生眼里，我是那种80％会喜欢，20％也不会讨厌的女生。只可惜自己属于晚熟型品种，又兼具追求完美的劣性。番然醒悟时，身边优秀的男生已被瓜分殆尽。现在的生活和学习空间相对狭窄，又无太多的时间广泛社交。所以网络交友成了我的一份希望。我来到这里，是因为大家都是热爱生活，想寻找爱情的人，彼此交往应以真诚沟通和相互信任为基础。我希望能在这里邂逅美丽的爱情，找到一生的幸福。理想中的他应该是：智慧善良、幽默大度、积极上进、乐观、有担当、足够真诚的男士，嗯,就是这样。最后,感谢浏览我资料和关注我的朋友祝福你们早日找到心仪的伴侣祝大家好运！！！', 'income': 2.0, 'horoscope': '双子座', 'blood_type': 'B', 'profession': 26.0, 'hometown_province': '湖南', 'hometown_city': '常德', 'ethnic': 56.0}}, {'_index': 'test_inde

#### 精准查询
term: 单一词语查询分词后结果，质量与分词有关;
terms: 结果是多个匹配词的结果的合集;

In [157]:
# keyword字段
res = es.search(index="test_index",  query={'term':{'nick':'我心'}})
print("A:", res)
res = es.search(index="test_index",  query={'term':{'nick':'我心飞翔'}})
print("B:", res)

A: {'took': 1, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}
B: {'took': 1, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 1, 'relation': 'eq'}, 'max_score': 4.591747, 'hits': [{'_index': 'test_index', '_id': '49575145', '_score': 4.591747, '_source': {'nick': '我心飞翔', 'gender': 'male', 'marry_state': 0, 'height': 173, 'weight': 90, 'education': 7, 'residence_city': '九龙坡', 'age': 47, 'interest': '[网络, 动物, 烹饪, 读书, 计算机, 中国菜, 健康食品, 励志与成功, 政治与军事, 哲学与宗教, 健康与养生, 自然科学, 动作冒险, 战争, 纪录片, 历史传记, 政治事件, 畅销书, 休闲生活, 行业发展, 饭店, 卡拉OK, 名山古刹, 风情名城, 高原雪域, 历史遗迹, 峻秀峡谷]', 'self_introduce': '真诚交友，非诚勿扰！介绍一下自己，我在一家国企工作，工作很忙，事情也很多，但相对来讲工作比较稳定，收入也还可以。这几年为了提升自己，又用周末时间重返校园攻读研究生硕士，所以这几年基本上是5+2、白+黑的状态，也耽误了婚姻大事。我属兔，农历是75年12月，公历是76年1月，个人有点小迷信，我希望最好能找属狗、猪、羊或其次是属兔、猴、虎的女朋友，同时我长得比较年轻，基本上看不出40多岁，所以希望女方年龄也不要超过38岁。我没有结过婚，也希望找个未婚的女性。异地恋也不太

In [165]:
# text字段
# term应该是分词后内容精准匹配
res = es.search(index="test_index",  query={'match':{'interest':'中国菜'}})
print("A:", res)
res = es.search(index="test_index",  query={'match':{'interest':'健康食品'}})
print("B:", res)
res = es.search(index="test_index",  query={'term':{'interest':'中国菜'}})
print("C:", res)
res = es.search(index="test_index",  query={'term':{'interest':'健康'}})
print("D", res)
res = es.search(index="test_index",  query={'term':{'interest':'食品'}})
print("E", res)
res = es.search(index="test_index",  query={'term':{'interest':'健康食品'}})
print("F:", res)
res = es.search(index="test_index",  query={'terms':{'interest':['中国菜', '健康']}})
print("I:", res)
res = es.search(index="test_index",  query={'terms':{'interest':['中国菜', '健康食品']}})
print("G:", res)

A: {'took': 4, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 125, 'relation': 'eq'}, 'max_score': 1.125726, 'hits': [{'_index': 'test_index', '_id': '314589592', '_score': 1.125726, '_source': {'nick': '雅', 'gender': 'female', 'marry_state': 0, 'height': 158, 'weight': 87, 'education': 5, 'residence_city': '长沙', 'age': 33, 'interest': '[影视, 音乐, 读书, 旅游, 中国菜, 文学]', 'self_introduce': '对世界充满好奇的钢琴老师，母胎单身至今，能感受到美好的事物，好希望能遇到一个人能一起分享眼前的繁花，心中的诗意畅想，真诚善良的两颗心携手同行，一起走在这蓝色星球的时光里.....', 'income': 2.0, 'horoscope': '天蝎座', 'hometown_province': '湖南', 'hometown_city': '常德', 'ethnic': 56.0}}, {'_index': 'test_index', '_id': '386595061', '_score': 1.1064631, '_source': {'nick': '紫芸腾', 'gender': 'female', 'marry_state': 0, 'height': 157, 'weight': 89, 'education': 5, 'residence_city': '广州', 'age': 40, 'interest': '[中国菜, 惬意海岛, 名山古刹, 秀美山水]', 'self_introduce': '我来自广东，平时喜欢爬山，打羽毛球。希望对方是以结婚为目的，期待与我有缘分的人出现，一起去创造浪漫之爱，共筑一个温馨的家。 静侯佳音，执子之手，与子偕老。注：有

In [39]:
# 存在字段查询
# 查询test索引中，包含age字段的数据
print(es.search(index='test_index',query={'exists':{'field':'age'}}))

{'took': 8, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 293, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '244389161', '_score': 1.0, '_source': {'nick': 'Panu', 'gender': 'male', 'marry_state': 0, 'height': 180, 'weight': 72, 'education': 8, 'residence_city': '海淀', 'age': 36, 'self_introduce': '我是芬兰数学家，在中科院工作. 我想找一个在北京工作和生活的女朋友.', 'income': 5.0, 'horoscope': '天秤座', 'blood_type': 'O', 'profession': 4.0, 'hometown_province': '其他'}}, {'_index': 'test_index', '_id': '177929372', '_score': 1.0, '_source': {'nick': '不离不弃ever', 'gender': 'male', 'marry_state': 0, 'height': 175, 'weight': 99, 'education': 7, 'residence_city': '西城', 'age': 48, 'interest': '[网络, 读书, 体育运动, 旅游, 中国菜, 健康食品, 励志与成功, 行业发展, 惬意海岛, 繁华都市, 秀美山水, 江河大川, 小桥流水人家]', 'self_introduce': '北京名校本硕毕业，京户，西城有房（无贷）。别人评价人品好、靠谱、综合素质高。包容、幽默、孝顺、稳重、诚信，有责任心。休息时看书，健身，听音乐。注重饮食，健康，不吸烟不好酒，没有任何不良嗜好。自身条件不差，之所以至今未婚是因为当年从外地考到帝都一路打拼，主要精力放在学业工作上，也因为

## 查询

### 查询所有数据

In [198]:
def print_result(res):
    if isinstance(res, ObjectApiResponse):
        print(res['hits']['total'])
    

In [199]:
query = {
        "match_all":{}
    }
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 294, 'relation': 'eq'}


### 查询存在值的字段数据

In [258]:
# 存在字段查询
# 查询存在字段的值
res = es.search(index='test_index',query={'exists':{'field':'university'}})
print_result(res)
res = es.search(index='test_index',query={'exists':{'field':'blood_type'}})
print_result(res)

{'value': 110, 'relation': 'eq'}
{'value': 186, 'relation': 'eq'}


### 等于查询term和terms

In [203]:
query = {
        "term": {
            "nick":"岁月"
        }
    }
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 1, 'relation': 'eq'}


In [204]:
res['hits']['hits']

[{'_index': 'test_index',
  '_id': '391437935',
  '_score': 4.591747,
  '_source': {'nick': '岁月',
   'gender': 'male',
   'marry_state': 1,
   'height': 177,
   'weight': 52,
   'education': 5,
   'residence_city': '青岛',
   'age': 41,
   'self_introduce': '我容易相处，为人实在，低调做人，踏实做事，懂得照顾呵护自己的女人是我最大的优点。习惯换位思考，只是些许大男子主义，不过也仅限于很偶然的那么几天，算是男性每个月的生理期，俗称大姨夫。性格开朗温顺的我是你值得信任和依靠托付一生的人。',
   'income': 4.0,
   'horoscope': '双子座',
   'blood_type': 'O',
   'profession': 7.0,
   'hometown_province': '福建',
   'hometown_city': '厦门',
   'ethnic': 56.0}}]

In [206]:
query = {
        "terms": {
            "hometown_city":["厦门","福建厦门"]
        }
    }
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 3, 'relation': 'eq'}


### 包含查询
match: 匹配[字段]包含"xxx"关键字的数据 \
multi_match: 多个字段里匹配包含"xxx"关键字的数据

In [207]:
query = {
        "match": {
            "hometown_city":"厦门"
        }
    }
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 3, 'relation': 'eq'}


In [209]:
query = {
        "multi_match": {
            "query":"厦门",
            "fields":["hometown_city","residence_city"]
        }
    }
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 4, 'relation': 'eq'}


### 搜索所有ids

In [210]:
query = {
        "ids": {
            "values":[71120970, 197836311]
        }
    }
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 2, 'relation': 'eq'}


### 复合查询bool
bool有3类查询关系，must(都满足),should(其中一个满足),must_not(都不满足)

In [211]:
query = {
    "bool":{
        "must":[
            {"term":{
                "gender":"female"}
            },
            {"match":{
                "marry_state":0}
            }
        ],
        "should":[
            {
                "terms":{
                    "hometown_city":["厦门","福建厦门"]
                }},
            {
                "multi_match": {
                    "query":"深圳",
                    "fields":["hometown_city","residence_city"]
                    }
            }
        ]
    }
}
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 111, 'relation': 'eq'}


### 切片查询
选定范围参数from, size? 返回结果有问题

In [220]:
query = {
    "match_all":{}
    }
res = es.search(index="test_index", query=query, from_=100, size=3)
print_result(res)

{'value': 294, 'relation': 'eq'}


### 范围查询

In [218]:
query = {
        "range":{
            "age":{
                "gte":18,       # >=18
                "lte":30        # <=30
            }
        }
}
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 22, 'relation': 'eq'}


### 前缀查询

In [222]:
query = {
        "prefix":{
            "university":"中国"
        }
}
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 4, 'relation': 'eq'}


### 通配符


In [225]:
query = {
        "wildcard":{
            "interest":"*菜"
        }
}
res = es.search(index="test_index", query=query)
print_result(res)

{'value': 125, 'relation': 'eq'}


### 正则查询

In [255]:
query = {
        "regexp":{
            "university":"中国.."
        }
}
res = es.search(index="test_index", query=query)
print_result(res)
print(res)


{'value': 2, 'relation': 'eq'}
{'took': 4, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '390612840', '_score': 1.0, '_source': {'nick': '阳光男孩', 'gender': 'male', 'marry_state': 0, 'height': 175, 'weight': 70, 'education': 7, 'residence_city': '成都', 'age': 34, 'interest': '[汽车, 读书, 体育运动, 旅游, 中国菜, 意大利菜, 健康食品, 励志与成功, 运动健身, 烹饪与饮食, 旅游, 爱情, 悬疑推理, 历史传记, 体育赛事, 理财投资, 影视热片, 电影院, 图书馆/书店, 体育馆, 逛街, 惬意海岛, 繁华都市, 历史遗迹]', 'self_introduce': '本人阳光爱笑类型，真诚可靠，来这里真诚找结婚对象，从小家教严格，不抽烟，不喝酒，不去酒吧夜店，没有什么不良嗜好，工作之余喜欢旅游、运动、看书等，希望缘分尽快到来。那些骗子请绕道，各种投资、借钱、酒托等都是骗子，请远离我！晚点遇到你，余生都是你。祝福大家早日找到自己的心仪对象！', 'income': 2.0, 'horoscope': '处女座', 'blood_type': 'A', 'profession': 8.0, 'ethnic': 56.0, 'university': '中国人民大学'}}, {'_index': 'test_index', '_id': '298984226', '_score': 1.0, '_source': {'nick': '未央', 'gender': 'female', 'marry_state': 1, 'height': 165, 'weight': 100, 'educatio

### 排序
asc升序，desc降序 \
多字段排序，注意顺序！写在前面的优先排序

In [226]:
query = {
        "match_all":{}
}
sort = [{
    "age":{"order":"asc"}
    },
    {
        "height":{
            "order":"asc"
        }
    }
]
res = es.search(index="test_index", query=query, sort=sort)
print_result(res)

{'value': 294, 'relation': 'eq'}


In [228]:
for per in res['hits']['hits']:
    print(per)

{'_index': 'test_index', '_id': '387991466', '_score': None, '_source': {'nick': '湘南*番禺*古坝*东明', 'gender': 'male', 'marry_state': 0, 'height': 175, 'weight': 53, 'education': 4, 'residence_city': '广州', 'age': 22, 'interest': '[网络, 体育运动, 中国菜, 艺术与摄影, 运动健身, 喜剧, 电影院, 图书馆/书店]', 'self_introduce': '来自湖南衡阳的我.学历不高.大专毕业.浓眉单眼皮可爱小眼睛.简单,孝顺,内敛.不抽烟.不喝酒.不赌博.平时爱好游泳和打羽毛球.来到广州这个大城市.一切都要从头开始.不像在家.都有父母的关爱.来到这个平台.希望找个能同甘共苦的.一起拼搏.志同道合的人.信息虚假和找借口投资的人还请绕道.勤奋的我一直努力着，终于可以让父母因我而骄傲。但是，再坚强的我，也会有疲惫的时候，多么希望早上起床，有人为我准备了早餐；多么期待晚上下班，有盏灯在为我等候着。期待能够与一位温…非诚勿扰', 'income': 1.0, 'horoscope': '处女座', 'blood_type': 'A', 'profession': 23.0, 'hometown_province': '湖南', 'hometown_city': '衡阳', 'ethnic': 56.0, 'university': '湖南铁路科技职业技术学院'}, 'sort': [22, 175]}
{'_index': 'test_index', '_id': '307291178', '_score': None, '_source': {'nick': '醉易温柔的在纽伦堡', 'gender': 'female', 'marry_state': 0, 'height': 175, 'weight': 69, 'education': 4, 'residence_city': '广州', 'age': 23, 'interest': '[舞蹈, 中国菜, 运动健身, 爱情, 娱乐八卦, 在自己或朋友家里, 繁华都市]', 'self_introd

### 响应过滤
filter_path, 多个条件用逗号隔开
es.search(index="index_name",doc_type="type_name",filter_path=["hits.hits._id"])

In [235]:
query = {
        "range":{
            "age":{
                "gte":18,       # >=18
                "lte":30        # <=30
            }
        }
}
res = es.search(index="test_index", query=query)
print(res)

{'took': 2, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 22, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '81312178', '_score': 1.0, '_source': {'nick': '南城阿牛', 'gender': 'male', 'marry_state': 0, 'height': 178, 'weight': 53, 'education': 5, 'residence_city': '南京', 'age': 30, 'interest': '[摄影, 音乐, 中国菜, 悬疑推理, 商场, 电影院, 逛街, 惬意海岛, 繁华都市, 风情名城, 历史遗迹]', 'self_introduce': '我是南京人 93年双鱼座 身高178 IT男。爱好：摄影、健身、弹吉他这个年纪应该为了自己的终生大事考虑考虑了，所以真心寻求一位我的另一半，希望你性格温顺，有自己的想法，如果肤白又爱笑，就更加分啦。我是个浪漫随性的人，生活是现实的，也是多彩的，我需要你陪我游遍千山万水，你需要我陪你度过春夏秋冬。', 'income': 2.0, 'horoscope': '双鱼座', 'profession': 3.0, 'hometown_province': '江苏', 'hometown_city': '南京', 'ethnic': 56.0}}, {'_index': 'test_index', '_id': '390589656', '_score': 1.0, '_source': {'nick': 'mayday', 'gender': 'male', 'marry_state': 0, 'height': 179, 'weight': 46, 'education': 3, 'residence_city': '沙坪坝', 'age': 29, 'self_introduce': '来自沙坪坝94年的双鱼座，也是很典型的双鱼座、现目前在家

In [236]:
res = es.search(index="test_index", query=query, filter_path=["hits.hits"])
print(res)


{'hits': {'hits': [{'_index': 'test_index', '_id': '81312178', '_score': 1.0, '_source': {'nick': '南城阿牛', 'gender': 'male', 'marry_state': 0, 'height': 178, 'weight': 53, 'education': 5, 'residence_city': '南京', 'age': 30, 'interest': '[摄影, 音乐, 中国菜, 悬疑推理, 商场, 电影院, 逛街, 惬意海岛, 繁华都市, 风情名城, 历史遗迹]', 'self_introduce': '我是南京人 93年双鱼座 身高178 IT男。爱好：摄影、健身、弹吉他这个年纪应该为了自己的终生大事考虑考虑了，所以真心寻求一位我的另一半，希望你性格温顺，有自己的想法，如果肤白又爱笑，就更加分啦。我是个浪漫随性的人，生活是现实的，也是多彩的，我需要你陪我游遍千山万水，你需要我陪你度过春夏秋冬。', 'income': 2.0, 'horoscope': '双鱼座', 'profession': 3.0, 'hometown_province': '江苏', 'hometown_city': '南京', 'ethnic': 56.0}}, {'_index': 'test_index', '_id': '390589656', '_score': 1.0, '_source': {'nick': 'mayday', 'gender': 'male', 'marry_state': 0, 'height': 179, 'weight': 46, 'education': 3, 'residence_city': '沙坪坝', 'age': 29, 'self_introduce': '来自沙坪坝94年的双鱼座，也是很典型的双鱼座、现目前在家附近上班，从事数控车工行业，有一套全款三室和代步车。长相6分左右 照片有点不上相 平时很爱听歌。自我标签是善良 对待感觉忠贞 有趣的灵魂。', 'income': 1.0, 'horoscope': '双鱼座', 'blood_type': 'A', 'ethnic': 56.0}}, {'_index': 'test

### 获取匹配数count


In [237]:
query = {
        "range":{
            "age":{
                "gte":18,       # >=18
                "lte":30        # <=30
            }
        }
}
res = es.count(index="test_index", query=query)
print(res)

{'count': 22, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}}


## 聚合

### 获取最小值

In [247]:
# 聚合查询
aggs = {
    "min_age": {
        "min": {
            "field": "age"
        }
    }
}

# 执行聚合查询
result = es.search(index="test_index", aggs= aggs)

min_age = result["aggregations"]["min_age"]["value"]
print(min_age)

22.0


### 最大值

In [248]:
# 聚合查询
aggs = {
    "max_age": {
        "max": {
            "field": "age"
        }
    }
}

# 执行聚合查询
result = es.search(index="test_index", aggs= aggs)

max_age = result["aggregations"]["max_age"]["value"]
print(max_age)

75.0


### 平均值

In [245]:
# 聚合查询
aggs = {
    "group_by_gender": {
        "terms": {
            "field": "gender"
        },
        "aggs": {
            "avg_age": {
                "avg": {
                    "field": "age"
                }
            }
        }
    }
}

# 执行聚合查询
# result = es.search(index="test_index", body={"size": 2, "aggs": aggs})
result = es.search(index="test_index", aggs= aggs)

# 输出结果
for bucket in result["aggregations"]["group_by_gender"]["buckets"]:
    print(bucket["key"], bucket["avg_age"]["value"])

female 44.09787234042553
male 42.58620689655172


### 求和

In [249]:
# 聚合查询
aggs = {
    "sum_age": {
        "sum": {
            "field": "age"
        }
    }
}

# 执行聚合查询
result = es.search(index="test_index", aggs= aggs)

sum_age = result["aggregations"]["sum_age"]["value"]
print(sum_age)

12833.0


In [250]:
query = {
        "range":{
            "age":{
                "gte":18,       # >=18
                "lte":30        # <=30
            }
        }
}

# 执行聚合查询
result = es.search(index="test_index", query=query)
print(result)


{'took': 2, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 22, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '81312178', '_score': 1.0, '_source': {'nick': '南城阿牛', 'gender': 'male', 'marry_state': 0, 'height': 178, 'weight': 53, 'education': 5, 'residence_city': '南京', 'age': 30, 'interest': '[摄影, 音乐, 中国菜, 悬疑推理, 商场, 电影院, 逛街, 惬意海岛, 繁华都市, 风情名城, 历史遗迹]', 'self_introduce': '我是南京人 93年双鱼座 身高178 IT男。爱好：摄影、健身、弹吉他这个年纪应该为了自己的终生大事考虑考虑了，所以真心寻求一位我的另一半，希望你性格温顺，有自己的想法，如果肤白又爱笑，就更加分啦。我是个浪漫随性的人，生活是现实的，也是多彩的，我需要你陪我游遍千山万水，你需要我陪你度过春夏秋冬。', 'income': 2.0, 'horoscope': '双鱼座', 'profession': 3.0, 'hometown_province': '江苏', 'hometown_city': '南京', 'ethnic': 56.0}}, {'_index': 'test_index', '_id': '390589656', '_score': 1.0, '_source': {'nick': 'mayday', 'gender': 'male', 'marry_state': 0, 'height': 179, 'weight': 46, 'education': 3, 'residence_city': '沙坪坝', 'age': 29, 'self_introduce': '来自沙坪坝94年的双鱼座，也是很典型的双鱼座、现目前在家

In [251]:
# 聚合查询
aggs = {
    "min_age": {
        "min": {
            "field": "age"
        }
    }
}

# 执行聚合查询
result = es.search(index="test_index", query=query, aggs=aggs)
print(result)
min_age = result["aggregations"]["min_age"]["value"]
print(min_age)

{'took': 4, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 22, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '81312178', '_score': 1.0, '_source': {'nick': '南城阿牛', 'gender': 'male', 'marry_state': 0, 'height': 178, 'weight': 53, 'education': 5, 'residence_city': '南京', 'age': 30, 'interest': '[摄影, 音乐, 中国菜, 悬疑推理, 商场, 电影院, 逛街, 惬意海岛, 繁华都市, 风情名城, 历史遗迹]', 'self_introduce': '我是南京人 93年双鱼座 身高178 IT男。爱好：摄影、健身、弹吉他这个年纪应该为了自己的终生大事考虑考虑了，所以真心寻求一位我的另一半，希望你性格温顺，有自己的想法，如果肤白又爱笑，就更加分啦。我是个浪漫随性的人，生活是现实的，也是多彩的，我需要你陪我游遍千山万水，你需要我陪你度过春夏秋冬。', 'income': 2.0, 'horoscope': '双鱼座', 'profession': 3.0, 'hometown_province': '江苏', 'hometown_city': '南京', 'ethnic': 56.0}}, {'_index': 'test_index', '_id': '390589656', '_score': 1.0, '_source': {'nick': 'mayday', 'gender': 'male', 'marry_state': 0, 'height': 179, 'weight': 46, 'education': 3, 'residence_city': '沙坪坝', 'age': 29, 'self_introduce': '来自沙坪坝94年的双鱼座，也是很典型的双鱼座、现目前在家

In [252]:
# 聚合查询
aggs = {
    "max_age": {
        "max": {
            "field": "age"
        }
    }
}
# 执行聚合查询
result = es.search(index="test_index", query=query, aggs=aggs)
print(result)
max_age = result["aggregations"]["max_age"]["value"]
print(max_age)

{'took': 3, 'timed_out': False, '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 22, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'test_index', '_id': '81312178', '_score': 1.0, '_source': {'nick': '南城阿牛', 'gender': 'male', 'marry_state': 0, 'height': 178, 'weight': 53, 'education': 5, 'residence_city': '南京', 'age': 30, 'interest': '[摄影, 音乐, 中国菜, 悬疑推理, 商场, 电影院, 逛街, 惬意海岛, 繁华都市, 风情名城, 历史遗迹]', 'self_introduce': '我是南京人 93年双鱼座 身高178 IT男。爱好：摄影、健身、弹吉他这个年纪应该为了自己的终生大事考虑考虑了，所以真心寻求一位我的另一半，希望你性格温顺，有自己的想法，如果肤白又爱笑，就更加分啦。我是个浪漫随性的人，生活是现实的，也是多彩的，我需要你陪我游遍千山万水，你需要我陪你度过春夏秋冬。', 'income': 2.0, 'horoscope': '双鱼座', 'profession': 3.0, 'hometown_province': '江苏', 'hometown_city': '南京', 'ethnic': 56.0}}, {'_index': 'test_index', '_id': '390589656', '_score': 1.0, '_source': {'nick': 'mayday', 'gender': 'male', 'marry_state': 0, 'height': 179, 'weight': 46, 'education': 3, 'residence_city': '沙坪坝', 'age': 29, 'self_introduce': '来自沙坪坝94年的双鱼座，也是很典型的双鱼座、现目前在家

query = {
        "range":{
            "age":{
                "gte":18,       # >=18
                "lte":30        # <=30
            }
        }
}